# Import Target Time Series Dataset
* This notebook has the following processes:
    * Create IAM role
    * Create a dataset group
    * Create a schema for a dataset
    * Create the dataset
    * Attach the dataset to the dataset group
    * Create a dataset import job
    
    
* **About 3 mins may be elapsed**


In [1]:
import boto3
from time import sleep
import os
import pandas as pd
import json
import time
import pprint
import numpy as np

In [2]:
%store -r

## Parmeters

In [3]:
DATASET_FREQUENCY = "W" 
TIMESTAMP_FORMAT = "yyyy-MM-dd"

suffix = str(np.random.uniform())[4:9]

# Enter a project name
project = 'WalmartKaggle'
target_suffix = '_target'

target_datasetName= project+'DS' + suffix
target_datasetGroupName= project +'DSG'+ suffix

In [4]:
with open('/opt/ml/metadata/resource-metadata.json') as notebook_info:
     data = json.load(notebook_info)
     resource_arn = data['ResourceArn']
     region = resource_arn.split(':')[3]
print(region)

us-east-2


In [5]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create role
**Make sure that a role for SageMaker notebook instance has these policies attached such as AmazonSageMakerFullAccess, AmazonS3FullAccess, AmazonForecastFullAccess, IAMFullAccess**

In [6]:
iam = boto3.client("iam")

# Put the role name
role_name = "ForecastRoleWalmart" + suffix
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
          "Effect": "Allow",
          "Principal": {
            "Service": "forecast.amazonaws.com"
          },
          "Action": "sts:AssumeRole"
        }
    ]
}

create_role_response = iam.create_role(
    RoleName = role_name,
    AssumeRolePolicyDocument = json.dumps(assume_role_policy_document)
)

# AmazonPersonalizeFullAccess provides access to any S3 bucket with a name that includes "personalize" or "Personalize" 
# if you would like tåo use a bucket with a different name, please consider creating and attaching a new policy
# that provides read access to your bucket or attaching the AmazonS3ReadOnlyAccess policy to the role
policy_arn = "arn:aws:iam::aws:policy/AmazonForecastFullAccess"
iam.attach_role_policy(
    RoleName = role_name,
    PolicyArn = policy_arn
)

# Now add S3 support
iam.attach_role_policy(
    PolicyArn='arn:aws:iam::aws:policy/AmazonS3FullAccess',
    RoleName=role_name
)
time.sleep(60) # wait for a minute to allow IAM role policy attachment to propagate

role_arn = create_role_response["Role"]["Arn"]
print(role_arn)

arn:aws:iam::057716757052:role/ForecastRoleWalmart35988


## Create DatasetGroup

In [7]:
# Create the DatasetGroup
create_dataset_group_response = forecast.create_dataset_group(
      DatasetGroupName= target_datasetGroupName,
      Domain="CUSTOM",
     )
target_datasetGroupArn = create_dataset_group_response['DatasetGroupArn']

In [8]:
forecast.describe_dataset_group(DatasetGroupArn=target_datasetGroupArn)

{'DatasetGroupName': 'WalmartKaggleDSG35988',
 'DatasetGroupArn': 'arn:aws:forecast:us-east-2:057716757052:dataset-group/WalmartKaggleDSG35988',
 'DatasetArns': [],
 'Domain': 'CUSTOM',
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 4, 15, 10, 9, 838000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 15, 10, 9, 838000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '6ece861a-51d5-4b49-9a22-e3e2a94deaeb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 15:10:09 GMT',
   'x-amzn-requestid': '6ece861a-51d5-4b49-9a22-e3e2a94deaeb',
   'content-length': '265',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Create schema

In [9]:
# Specify the schema of your dataset here. Make sure the order of columns matches the raw data files.
schema ={
   "Attributes":[
      {
         "AttributeName":"timestamp",
         "AttributeType":"timestamp"
      },
      {
         "AttributeName":"target_value",
         "AttributeType":"float"
      },
      {
         "AttributeName":"item_id",
         "AttributeType":"string"
      }
   ]
}

## Create Target Time Sereis Dataset

In [10]:
response=forecast.create_dataset(
                    Domain="CUSTOM",
                    DatasetType='TARGET_TIME_SERIES',
                    DatasetName=target_datasetName,
                    DataFrequency=DATASET_FREQUENCY, 
                    Schema = schema
)

In [11]:
target_datasetArn = response['DatasetArn']
forecast.describe_dataset(DatasetArn=target_datasetArn)

{'DatasetArn': 'arn:aws:forecast:us-east-2:057716757052:dataset/WalmartKaggleDS35988',
 'DatasetName': 'WalmartKaggleDS35988',
 'Domain': 'CUSTOM',
 'DatasetType': 'TARGET_TIME_SERIES',
 'DataFrequency': 'W',
 'Schema': {'Attributes': [{'AttributeName': 'timestamp',
    'AttributeType': 'timestamp'},
   {'AttributeName': 'target_value', 'AttributeType': 'float'},
   {'AttributeName': 'item_id', 'AttributeType': 'string'}]},
 'EncryptionConfig': {},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 4, 4, 15, 10, 9, 984000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 4, 4, 15, 10, 9, 984000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c242d60e-8112-4282-89db-b014c4a8011b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 15:10:09 GMT',
   'x-amzn-requestid': 'c242d60e-8112-4282-89db-b014c4a8011b',
   'content-length': '501',
   'connection': 'keep-alive'},
  'RetryAttempts'

## Attach the target time series dataset to the DatasetGroup

In [12]:
# Attach the Dataset to the Dataset Group:
forecast.update_dataset_group(
    DatasetGroupArn=target_datasetGroupArn, 
    DatasetArns=[target_datasetArn])

{'ResponseMetadata': {'RequestId': '4fa22216-020e-42d3-9283-3004718bdc72',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sat, 04 Apr 2020 15:10:09 GMT',
   'x-amzn-requestid': '4fa22216-020e-42d3-9283-3004718bdc72',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

## Create a bucket

In [13]:
import boto3
import sagemaker

s3_resource = boto3.resource('s3')
s3 = boto3.client('s3')

# if you want, replace with a name of your S3 bucket
bucket_name = sagemaker.Session().default_bucket()  

if s3_resource.Bucket(bucket_name).creation_date is None:
    # bucket is not existing 
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})    
else: 
    # Bucket exists
    print("bucket name is ", bucket_name)
    

bucket name is  sagemaker-us-east-2-057716757052


### Upload target data to S3

In [14]:
# Upload Target File under a bucket folder
bucket_folder = project
s3_file_path = bucket_folder + "/" + target_time_series_filename

boto3.Session().resource('s3').Bucket(bucket_name).Object(s3_file_path).upload_file(target_time_series_path)
target_s3DataPath = "s3://"+bucket_name + "/" + s3_file_path

## Create dataset_import_job used to download dataset from S3

In [16]:
# Finally we can call import the dataset
datasetImportJobName = 'DSIMPORT_JOB_TARGET_WALMART' + suffix
ds_import_job_response=forecast.create_dataset_import_job(DatasetImportJobName=datasetImportJobName,
                                                          DatasetArn=target_datasetArn,
                                                          DataSource= {
                                                              "S3Config" : {
                                                                 "Path":target_s3DataPath,
                                                                 "RoleArn": role_arn
                                                              } 
                                                          },
                                                          TimestampFormat=TIMESTAMP_FORMAT
                                                         )

In [17]:
ds_target_import_job_arn=ds_import_job_response['DatasetImportJobArn']
print(ds_target_import_job_arn)

arn:aws:forecast:us-east-2:057716757052:dataset-import-job/WalmartKaggleDS35988/DSIMPORT_JOB_TARGET_WALMART35988


In [18]:
%%time

while True:
    dataImportStatus = forecast.describe_dataset_import_job(DatasetImportJobArn=ds_target_import_job_arn)['Status']
    print(dataImportStatus)
    if dataImportStatus != 'ACTIVE' and dataImportStatus != 'CREATE_FAILED':
        sleep(30)
    else:
        break

CREATE_PENDING
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
CREATE_IN_PROGRESS
ACTIVE
CPU times: user 47.3 ms, sys: 0 ns, total: 47.3 ms
Wall time: 5min 30s


In [19]:
%store project
%store suffix
%store target_suffix
%store region
%store ds_target_import_job_arn
%store target_datasetArn
%store target_datasetGroupArn
%store bucket_name
%store bucket_folder
%store role_arn
%store role_name


Stored 'project' (str)
Stored 'suffix' (str)
Stored 'target_suffix' (str)
Stored 'region' (str)
Stored 'ds_target_import_job_arn' (str)
Stored 'target_datasetArn' (str)
Stored 'target_datasetGroupArn' (str)
Stored 'bucket_name' (str)
Stored 'bucket_folder' (str)
Stored 'role_arn' (str)
Stored 'role_name' (str)
